# Open Source Software Development for Reliability and Lifetime Calculation
## TUMWVIB Chair of Vibroacoustics of Vehicles and Machines
#### Supervisors:  
* Sepahvand, Kheirollah; Dr.-Ing. habil.
* Kreuter, Daniel Christopher; Dr.-Ing
* Mueller, Johannes; Dr.-Ing

#### Student: 
Mustapha Kassem

## Bayes evaluation tool

In [1]:
import pymc3 as pm
import numpy.ma as ma
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np
import pandas as pd
import theano
theano.gof.compilelock.set_lock_status(False)
from scipy import stats
import theano
import theano.tensor as tt
import mystic as my
from os import path
import io
import sys, os
import json

sys.path.insert(0, os.path.abspath('..'))

from pylife.materialdata.woehler.fatigue_data import FatigueData
from pylife.materialdata.woehler.woehler_curve_creator import WoehlerCurveCreator
from pylife.materialdata.woehler.utils.bayes_analyzer import *
from pylife.materialdata.woehler.loglike import *

#data = pd.read_excel("C:/Users/")
data = pd.read_excel('../pylife/materialdata/data/Test_dat.xlsx')
data.columns=['loads', 'cycles']

# Limit of the load cycle
ld_cyc_lim = data['cycles'].max()
#ld_cyc_lim = 10e6


# Class WoehlerCurve
fatigue_data = FatigueData(data, ld_cyc_lim)
woehler_curve_creator = WoehlerCurveCreator(fatigue_data)
woehler_curve = woehler_curve_creator.maximum_like_procedure({})
x = np.log10(fatigue_data.fractures.loads)
y = np.log10(fatigue_data.fractures.cycles)
bayes_analyzer = BayesAnalyzer()

slope_full, intercept_full, r_value, p_value, std_err = stats.linregress(x, y)


Optimization terminated successfully.
         Current function value: 152.855959
         Iterations: 458
         Function evaluations: 654


In [2]:
test_runs = 1
n_N=[0,0,0,0,0,0,0,5,0,5,0,5,0,5]
n_N_inf=[3,0,3,0,3,0,0]
np.sum(n_N)+ np.sum(n_N_inf), fatigue_data.data.loads.shape[0]

(29, 452)

In [ ]:
step = 0
for n in range(test_runs):
    step = step + 1 
#### DATA
    ndraws = 5000  # number of draws from the distribution
    nburn = 1000   # number of "burn-in points" (which we'll discard)
    
    data_small_fin = bayes_analyzer.ld_lvl_sample(woehler_curve, data, n_N)
    data_small_inf, data_inf_dict, n_frac = bayes_analyzer.ld_lvl_sample_inf(woehler_curve, n_N_inf)

    data_total_x = np.concatenate((data_small_fin['x'], data_small_inf['x']))
    data_total_y = np.concatenate((data_small_fin['y'], data_small_inf['y']))
    data_total = {'x':10**data_total_x, 'y':10**data_total_y}

    dataset = pd.DataFrame(data_total)
    dataset.columns=['loads', 'cycles']
    fatigue_data_sub = FatigueData(dataset, ld_cyc_lim)
    woehler_curve_creator_sub = WoehlerCurveCreator(fatigue_data_sub)
    woehler_curve_sub = woehler_curve_creator_sub.maximum_like_procedure({})
    x_sub = np.log10(fatigue_data_sub.fractures.loads)
    y_sub = np.log10(fatigue_data_sub.fractures.cycles)
    data_small_frac = dict(x=x_sub, y=y_sub)

    slope_sub, intercept_sub, r_value, p_value, std_err = stats.linregress(x_sub, y_sub)
    
#### k    
    with pm.Model() as model_robust:
        trace_robust = bayes_analyzer.bayesian_slope_norm(data_dict=data_small_frac, model_name=model_robust, 
                                           samples=ndraws, chains=2)

    burned_trace_robust = trace_robust[1000:]
    pm.traceplot(burned_trace_robust)

#### TN
    with pm.Model() as WL:
        stdev = pm.HalfNormal('stdev', sd=1.3) #sd standard wert (log-normal/ beat Verteilung/exp lambda)
        mu = pm.Normal('mu', mu=np.log10(fatigue_data_sub.N_shift).mean(), sd=np.log10(fatigue_data_sub.N_shift).std()) #mu könnte von FKM gegeben

        y = pm.Normal('y', mu=mu, sd=stdev, observed=np.log10(fatigue_data_sub.N_shift)) #lognormal

        trace_TN = pm.sample(ndraws, nuts_kwargs={'target_accept': 0.99}, chains=3, tune=1000)
        
    burned_trace_TN = trace_TN[1000:]
    #pm.traceplot(burned_trace_TN)
    
#### SD, TS    
    data_S = data_small_inf['x']
    data_N = data_small_inf['y']

    # create our Op
    logl = LogLike(mali_sum_lolli, data_S, data_N, fatigue_data_sub.load_cycle_limit)

    # use PyMC3 to sampler from log-likelihood
    with pm.Model():
        # set priors on model gradient and y-intercept
        SD = pm.Normal('SD_50', mu=data_S.mean(), sd=data_S.std()*5)
        #TS = pm.Exponential('TS_50', lam=1.1*2.5631031311)
        TS = pm.Lognormal('TS_50', mu=np.log10(1.1), sd=np.log10(0.5))

        # convert m and c to a tensor vector
        var = tt.as_tensor_variable([SD, TS])

        # use a DensityDist (use a lamdba function to "call" the Op)
        '''pm.traceplot(trace)
            plt.show()
            doesnt work with density disk yet

            https://discourse.pymc.io/t/pm-traceplot-error/3524
        '''
        pm.DensityDist('likelihood', lambda v: logl(v), observed={'v': var})

        trace_SD_TS = pm.sample(ndraws, tune=nburn, chains=3, discard_tuned_samples=True)
        
#### DATA EXPORT
    if step==1:
        with open('bayes_dict_5.json', 'w') as fp:
            json.dump({'Bayes_TS':[trace_quantiles(trace_SD_TS)[0]['TS_50'][50]/2.5631031311],
                       'Mali_TS':[woehler_curve.curve_parameters['1/TS']],
                       'Bayes_SD':[trace_quantiles(trace_SD_TS)[0]['SD_50'][50]],
                       'Mali_SD':[np.log10(woehler_curve.curve_parameters['SD_50'])],
                       'Bayes_TN':[trace_quantiles(burned_trace_TN)[0]['stdev'][50]],
                       'Mali_TN':[woehler_curve.curve_parameters['1/TN']],
                       'Bayes_k':-[trace_quantiles(burned_trace_robust)[0]['x'][50]],
                       'Mali_k':[woehler_curve.curve_parameters['k_1']]},fp)
    else:
        with open('bayes_dict_5.json', 'r') as fp:
            dict_import = json.load(fp)
        with open('bayes_dict_5.json', 'w') as feedsjson:
            dict_import['Bayes_TS'].append(trace_quantiles(trace_SD_TS)[0]['TS_50'][50]/2.5631031311)
            dict_import['Bayes_SD'].append(trace_quantiles(trace_SD_TS)[0]['SD_50'][50])
            dict_import['Bayes_TN'].append(trace_quantiles(burned_trace_TN)[0]['mu'][50])
            dict_import['Bayes_k'].append(trace_quantiles(burned_trace_robust)[0]['x'][50])
            dict_import['Mali_TS'].append(woehler_curve_sub.curve_parameters['1/TS'])
            dict_import['Mali_SD'].append(np.log10(woehler_curve_sub.curve_parameters['SD_50']))
            dict_import['Mali_TN'].append(woehler_curve_sub.curve_parameters['1/TN'])
            dict_import['Mali_k'].append(woehler_curve_sub.curve_parameters['k_1'])

            json.dump(dict_import, feedsjson)

Optimization terminated successfully.
         Current function value: 10.673752
         Iterations: 662
         Function evaluations: 853


Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 4 jobs)
NUTS: [sd, x, Intercept]
Sampling 2 chains, 0 divergences:  20%|█▉        | 2375/12000 [01:24<07:51, 20.40draws/s]

In [ ]:
with open('bayes_dict.json', 'r') as fp:
    dict_import = json.load(fp)

fig = plt.figure(figsize=(7, 7))
sorted_data = dict_import['Bayes_TS'].sort()
mu = np.mean(dict_import['Bayes_TS'])
std = np.std(dict_import['Bayes_TS'])
ax = fig.add_subplot(111, xlabel='x', ylabel='y', title='Generated data and underlying model of 1/TS')
ax.plot(np.arange(len(dict_import['Bayes_TS'])),stats.norm.cdf(dict_import['Bayes_TS'], mu, std),  'g')
#plt.legend(loc=0)

In [ ]:
fig = plt.figure(figsize=(7, 7))
ax = fig.add_subplot(111, xlabel='x', ylabel='y', title='Generated data and underlying model of 1/TS')
ax.plot(np.arange(len(dict_import['Bayes_TS'])), 
        np.ones(len(dict_import['Mali_TS']))*np.mean(np.array([*dict_import['Bayes_TS']])*2.5631031311),  'g')
ax.plot( np.arange(len(dict_import['Mali_TS'])),
        np.ones(len(dict_import['Mali_TS']))*np.mean(dict_import['Mali_TS']), 'r')

ax.plot(np.arange(len(dict_import['Bayes_TS'])),np.array([*dict_import['Bayes_TS']])*2.5631031311,  'gx', label='Bayes_TS')
ax.plot( np.arange(len(dict_import['Mali_TS'])),dict_import['Mali_TS'], 'rx', label='Mali_TS')
ax.plot(np.arange(len(dict_import['Mali_TS'])),
        np.ones(len(dict_import['Mali_TS']))*woehler_curve.curve_parameters['1/TS'],'b', label='Full')
plt.legend(loc=0)

In [ ]:
fig = plt.figure(figsize=(7, 7))
ax = fig.add_subplot(111, xlabel='x', ylabel='y', title='Generated data and underlying model of SD')
ax.plot(np.arange(len(dict_import['Bayes_SD'])), 
        np.ones(len(dict_import['Mali_SD']))*np.mean(dict_import['Bayes_SD']),  'g')
ax.plot( np.arange(len(dict_import['Mali_SD'])),
        np.ones(len(dict_import['Mali_SD']))*np.mean(dict_import['Mali_SD']), 'r')

ax.plot(np.arange(len(dict_import['Bayes_SD'])), dict_import['Bayes_SD'],  'gx', label='Bayes_SD')
ax.plot(np.arange(len(dict_import['Mali_SD'])),dict_import['Mali_SD'], 'rx', label='Mali_SD')
ax.plot(np.arange(len(dict_import['Mali_SD'])),
        np.ones(len(dict_import['Mali_SD']))*np.log10(woehler_curve.curve_parameters['SD_50']),'b', label='Full')
plt.legend(loc=0)

In [ ]:
fig = plt.figure(figsize=(7, 7))
ax = fig.add_subplot(111, xlabel='x', ylabel='y', title='Generated data and underlying model of 1/TN')
ax.plot(np.arange(len(dict_import['Bayes_TN'])), dict_import['Bayes_TN'],  'gx', label='Bayes_TN')
ax.plot( np.arange(len(dict_import['Mali_TN'])),dict_import['Mali_TN'], 'rx', label='Mali_TN')

ax.plot(np.arange(len(dict_import['Bayes_TN'])), 
        np.ones(len(dict_import['Mali_TN']))*np.mean(dict_import['Bayes_TN']),  'g')
ax.plot( np.arange(len(dict_import['Mali_TN'])),
        np.ones(len(dict_import['Mali_TN']))*np.mean(dict_import['Mali_TN']), 'r')
ax.plot(np.arange(len(dict_import['Mali_TN'])),
        np.ones(len(dict_import['Mali_TN']))*woehler_curve.curve_parameters['1/TN'],'b', 
        label='Full %f'%woehler_curve.curve_parameters['1/TN'])
#ax.set_yscale('log')
plt.legend(loc=0);

In [ ]:
fig = plt.figure(figsize=(7, 7))
ax = fig.add_subplot(111, xlabel='x', ylabel='y', title='Generated data and underlying model of slope k')
ax.plot(np.arange(len(dict_import['Bayes_k'])), 
        dict_import['Bayes_k']*np.ones(len(dict_import['Bayes_k']))*-1,  'gx', label='Bayes_k')
ax.plot( np.arange(len(dict_import['Mali_k'])),dict_import['Mali_k'], 'rx', label='Mali_k')

ax.plot(np.arange(len(dict_import['Bayes_k'])), 
        np.mean(dict_import['Bayes_k'])*np.ones(len(dict_import['Bayes_k']))*-1,  'g')
ax.plot( np.arange(len(dict_import['Mali_k'])),
        np.mean(dict_import['Mali_k'])*np.ones(len(dict_import['Bayes_k'])), 'r')
ax.plot(np.arange(len(dict_import['Mali_k'])),
        np.ones(len(dict_import['Mali_k']))*woehler_curve.curve_parameters['k_1'],'b', label='Full')
plt.legend(loc=0);

    with open('bayes_dict.json', 'w') as fp:
        json.dump({'Bayes_TS':[trace_quantiles(trace_SD_TS)[0]['TS_50'][50]/2.5631031311],
                   'Mali_TS':[woehler_curve_sub.curve_parameters['1/TS']],
                   'Bayes_SD':[trace_quantiles(trace_SD_TS)[0]['SD_50'][50]],
                   'Mali_SD':[np.log10(woehler_curve_sub.curve_parameters['SD_50'])],
                   'Bayes_TN':[trace_quantiles(burned_trace_TN)[0]['stdev'][50]],
                   'Mali_TN':[woehler_curve_sub.curve_parameters['1/TN']],
                   'Bayes_k':-[trace_quantiles(burned_trace_robust)[0]['x'][50]],
                   'Mali_k':[woehler_curve_sub.curve_parameters['k_1']]},fp)
                   
    with open('bayes_dict_5.json', 'r') as fp:
        dict_import = json.load(fp)
    with open('bayes_dict.json', 'w') as feedsjson:
        dict_import['Bayes_TS'].append(trace_quantiles(trace_SD_TS)[0]['TS_50'][50]/2.5631031311)
        dict_import['Bayes_SD'].append(trace_quantiles(trace_SD_TS)[0]['SD_50'][50])
        dict_import['Bayes_TN'].append(trace_quantiles(burned_trace_TN)[0]['stdev'][50])
        dict_import['Bayes_k'].append(trace_quantiles(burned_trace_robust)[0]['x'][50])
        dict_import['Mali_TS'].append(woehler_curve_sub.curve_parameters['1/TS'])
        dict_import['Mali_SD'].append(np.log10(woehler_curve_sub.curve_parameters['SD_50']))
        dict_import['Mali_TN'].append(woehler_curve_sub.curve_parameters['1/TN'])
        dict_import['Mali_k'].append(woehler_curve_sub.curve_parameters['k_1'])

        json.dump(dict_import, feedsjson)